
# YOLOv8 Binary Object Detection and Cropping Pipeline

This notebook includes:

1. Converting YOLO dataset annotations from multi-class to binary.
2. Fine-tuning YOLOv8 for binary detection.
3. Inference pipeline (detection -> cropping -> augmentation -> classification preparation).

Abdulmlik almuhanna

In [1]:

!pip install -q ultralytics roboflow
!pip install -q glob2
import os

from ultralytics import YOLO
from roboflow import Roboflow
import cv2
import numpy as np
from glob import glob


In [2]:

from roboflow import Roboflow
rf = Roboflow(api_key="TS1niacLXvvWTierCKCT")
project = rf.workspace("insectai").project("insects_noclasses")
version = project.version(7)
dataset = version.download("yolov8")



loading Roboflow workspace...
loading Roboflow project...


### create negative images

In [15]:
import os
import cv2
import numpy as np

# === Paths ===
image_folder = '/content/insects_noclasses-7/train/images'
label_folder = '/content/insects_noclasses-7/train/labels'
output_folder = 'negative_images_inpainted'
os.makedirs(output_folder, exist_ok=True)

def yolo_to_bbox(yolo_bbox, img_width, img_height):
    x_center, y_center, w, h = yolo_bbox
    x_center *= img_width
    y_center *= img_height
    w *= img_width
    h *= img_height
    x1 = int(x_center - w / 2)
    y1 = int(y_center - h / 2)
    x2 = int(x_center + w / 2)
    y2 = int(y_center + h / 2)
    return x1, y1, x2, y2

for filename in os.listdir(image_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(image_folder, filename)
        label_path = os.path.join(label_folder, os.path.splitext(filename)[0] + '.txt')
        output_path = os.path.join(output_folder, filename)

        img = cv2.imread(image_path)
        if img is None:
            continue
        h, w = img.shape[:2]

        mask = np.zeros((h, w), dtype=np.uint8)

        # Create mask from bbox annotations
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) != 5:
                        continue
                    _, *bbox = map(float, parts)
                    x1, y1, x2, y2 = yolo_to_bbox(bbox, w, h)
                    cv2.rectangle(mask, (x1, y1), (x2, y2), 255, -1)  # White mask

            # Inpaint using nearby pixels
            inpainted = cv2.inpaint(img, mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)
            cv2.imwrite(output_path, inpainted)

print("✅ Done! Inpainted images saved to:", output_folder)


✅ Done! Inpainted images saved to: negative_images_inpainted


### Load Negative images to our Train folder


In [3]:
!mkdir negs

mkdir: cannot create directory ‘negs’: File exists


In [13]:
import os
import shutil

# === INPUT CONFIG ===
source_folder = '/hpc/home/ama191/Our_Yolo_Solution/negs'   # Folder with negative images
target_folder = '/hpc/home/ama191/Our_Yolo_Solution/insects_noclasses-7/train/images'        # Your YOLO training images folder

# === Ensure target folder exists ===
os.makedirs(target_folder, exist_ok=True)

# === Loop through and copy only image files ===
image_extensions = ('.jpg', '.jpeg', '.png')

for filename in os.listdir(source_folder):
    if filename.lower().endswith(image_extensions):
        src_path = os.path.join(source_folder, filename)
        dst_path = os.path.join(target_folder, filename)
        shutil.copy2(src_path, dst_path)  # use shutil.move if you want to move instead of copy

print(f"✅ {len(os.listdir(source_folder))} negative images copied to: {target_folder}")


✅ 21 negative images copied to: /hpc/home/ama191/Our_Yolo_Solution/insects_noclasses-7/train/images


### this is for removing classes but we dont need it now


In [ ]:

import glob

label_paths = glob.glob(f"{dataset.location}/train/labels/*.txt")

for label_path in label_paths:
    with open(label_path, "r") as file:
        lines = file.readlines()

    with open(label_path, "w") as file:
        for line in lines:
            parts = line.strip().split()
            parts[0] = '0'  # Binary class (0: insect)
            file.write(' '.join(parts) + '\n')

label_paths2 = glob.glob(f"{dataset.location}/valid/labels/*.txt")

for label_path in label_paths2:
    with open(label_path, "r") as file:
        lines = file.readlines()

    with open(label_path, "w") as file:
        for line in lines:
            parts = line.strip().split()
            parts[0] = '0'  # Binary class (0: insect)
            file.write(' '.join(parts) + '\n')
print("Converted labels to binary classes successfully.")


Converted labels to binary classes successfully.


In [ ]:
!pip install torch
import torch
import gc
torch.cuda.empty_cache()
gc.collect()

Defaulting to user installation because normal site-packages is not writeable


498

### creating a pose Dataset


In [10]:
from pathlib import Path

labels = glob("/hpc/home/ama191/Our_Yolo_Solution/insects_noclasses-7/valid/labels/*.txt", recursive=True)

root_dir = Path(labels[0]).parent.parent.parent
for label in labels:
    label = Path(label)
    split = label.parent.parent.name
    img_path = [(root_dir / split / "images" / label.with_suffix(sfx).name)
                for sfx in [".png", ".jpg", ".PNG", ".JPG"]]
    img_path = [pth for pth in img_path if pth.exists()][0]
    boxes = []
    points = []
    classes = []
    save_pth = root_dir / split / "labels_kp" / label.name
    save_pth.parent.mkdir(exist_ok=True)
    with open(label) as f:
        lines = f.readlines()
        for line in lines:
            splits = line.rstrip().split(" ")
            cls_id = int(splits[0])
            box = splits[1:]
            if not box:
                with open(save_pth, "w") as f:
                    pass
                continue

            box = [float(pt) for pt in box]
            point = (box[0], box[1])
            points.append(point)
            boxes.append(box)
            classes.append(cls_id)

    with open(save_pth, "w") as f:
        for point, box, cls_id in zip(points, boxes, classes):
            f.writelines(f"{cls_id} {box[0]} {box[1]} {box[2]} {box[3]} {point[0]} {point[1]} 1 \n")

### training the binary detector

In [5]:
!git clone https://huggingface.co/davsolai/yolo11x-p2-coco


Cloning into 'yolo11x-p2-coco'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 11 (delta 0), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (11/11), 3.62 KiB | 21.00 KiB/s, done.


In [6]:
!cd yolo11x-p2-coco && git fetch origin refs/pr/1:pr/1

From https://huggingface.co/davsolai/yolo11x-p2-coco
 * [new ref]         refs/pr/1  -> pr/1


In [ ]:
!yolo cfg

In [14]:
from ultralytics import YOLO

# Initialize the YOLO model
model = YOLO("/hpc/home/ama191/Our_Yolo_Solution/yolo11x-p2-coco/model (1).pt")

# Define search space
search_space = {
    "lr0": (1e-5, 1e-1),
    "degrees": (0.0, 45.0),
    "dfl":(0.5,7)
}

# Tune hyperparameters on COCO8 for 30 epochs
model.tune(
    data="/hpc/home/ama191/Our_Yolo_Solution/insects_noclasses-7/data.yaml",
    epochs=30,
    iterations=300,
    optimizer="AdamW",
    space=search_space,
    plots=True,
    save=True,
    val=True,
    imgsz=1024
)

Tuner: Initialized Tuner instance with 'tune_dir=runs/detect/tune2'
Tuner: 💡 Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/300 with hyperparameters: {'lr0': 0.01, 'degrees': 0.0, 'dfl': 1.5}
New https://pypi.org/project/ultralytics/8.3.157 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.152 🚀 Python-3.11.11 torch-2.7.0+cu126 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/hpc/home/ama191/Our_Yolo_Solution/insects_noclasses-7/data.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False

Traceback (most recent call last):
  File "/hpc/home/ama191/.config/Ultralytics/DDP/_temp_u672v8fg139792836792080.py", line 13, in <module>
    results = trainer.train()
              ^^^^^^^^^^^^^^^
  File "/hpc/home/ama191/miniconda3/envs/test/lib/python3.11/site-packages/ultralytics/engine/trainer.py", line 227, in train
    self._do_train(world_size)
  File "/hpc/home/ama191/miniconda3/envs/test/lib/python3.11/site-packages/ultralytics/engine/trainer.py", line 347, in _do_train
    self._setup_ddp(world_size)
  File "/hpc/home/ama191/miniconda3/envs/test/lib/python3.11/site-packages/ultralytics/engine/trainer.py", line 239, in _setup_ddp
    torch.cuda.set_device(RANK)
  File "/hpc/home/ama191/miniconda3/envs/test/lib/python3.11/site-packages/torch/cuda/__init__.py", line 529, in set_device
    torch._C._cuda_setDevice(device)
RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrec

Overriding model.yaml nc=80 with nc=1


W0620 14:51:45.401000 2015958 site-packages/torch/distributed/elastic/multiprocessing/api.py:900] Sending process 2015979 closing signal SIGTERM
E0620 14:51:45.475000 2015958 site-packages/torch/distributed/elastic/multiprocessing/api.py:874] failed (exitcode: 1) local_rank: 1 (pid: 2015980) of binary: /hpc/home/ama191/miniconda3/envs/test/bin/python
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/hpc/home/ama191/miniconda3/envs/test/lib/python3.11/site-packages/torch/distributed/run.py", line 896, in <module>
    main()
  File "/hpc/home/ama191/miniconda3/envs/test/lib/python3.11/site-packages/torch/distributed/elastic/multiprocessing/errors/__init__.py", line 355, in wrapper
    return f(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^
  File "/hpc/home/ama191/miniconda3/envs/test/lib/python3.11/site-packages/torch/distributed/run.py", line 892, in main
    run(args)
  File "/hpc/home/a

ERROR ❌ training failure for hyperparameter tuning iteration 1
Command '['/hpc/home/ama191/miniconda3/envs/test/bin/python', '-m', 'ultralytics.cfg.__init__', 'train', 'task=detect', 'mode=train', 'model=/hpc/home/ama191/Our_Yolo_Solution/yolo11x-p2-coco/model (1).pt', 'data=/hpc/home/ama191/Our_Yolo_Solution/insects_noclasses-7/data.yaml', 'epochs=30', 'time=None', 'patience=100', 'batch=16', 'imgsz=1024', 'save=True', 'save_period=-1', 'cache=False', 'device=None', 'workers=8', 'project=None', 'name=None', 'exist_ok=False', 'pretrained=True', 'optimizer=AdamW', 'verbose=True', 'seed=0', 'deterministic=True', 'single_cls=False', 'rect=False', 'cos_lr=False', 'close_mosaic=10', 'resume=False', 'amp=True', 'fraction=1.0', 'profile=False', 'freeze=None', 'multi_scale=False', 'overlap_mask=True', 'mask_ratio=4', 'dropout=0.0', 'val=True', 'split=val', 'save_json=False', 'conf=None', 'iou=0.7', 'max_det=300', 'half=False', 'dnn=False', 'plots=True', 'source=None', 'vid_stride=1', 'stream_b

       1/30      13.9G      3.708      4.855      2.067       4537        640:  14%|█▍        | 1/7 [00:43<04:22, 43.78s/it]

WARNING ⚠️ CUDA OutOfMemoryError in TaskAlignedAssigner, using CPU
New https://pypi.org/project/ultralytics/8.3.157 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.152 🚀 Python-3.11.11 torch-2.7.0+cu126 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/hpc/home/ama191/Our_Yolo_Solution/insects_noclasses-7/data.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.00912, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, mod

W0620 14:51:58.848000 2015573 site-packages/torch/distributed/elastic/multiprocessing/api.py:900] Sending process 2015588 closing signal SIGTERM
E0620 14:51:59.271000 2015573 site-packages/torch/distributed/elastic/multiprocessing/api.py:874] failed (exitcode: -9) local_rank: 0 (pid: 2015587) of binary: /hpc/home/ama191/miniconda3/envs/test/bin/python
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/hpc/home/ama191/miniconda3/envs/test/lib/python3.11/site-packages/torch/distributed/run.py", line 896, in <module>
    main()
  File "/hpc/home/ama191/miniconda3/envs/test/lib/python3.11/site-packages/torch/distributed/elastic/multiprocessing/errors/__init__.py", line 355, in wrapper
    return f(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^
  File "/hpc/home/ama191/miniconda3/envs/test/lib/python3.11/site-packages/torch/distributed/run.py", line 892, in main
    run(args)
  File "/hpc/home/

YOLO11x-p2 summary: 440 layers, 57,788,468 parameters, 57,788,452 gradients, 244.1 GFLOPs

Transferred 1245/1253 items from pretrained weights
DDP: debug command /hpc/home/ama191/miniconda3/envs/test/bin/python -m torch.distributed.run --nproc_per_node 2 --master_port 34653 /hpc/home/ama191/.config/Ultralytics/DDP/_temp_y5z3kgcl140233061802320.py


KeyboardInterrupt: 

In [10]:
import yaml

# Define hyperparameters for the detector model
hyperparams = { # Replace this dynamically if needed
    'epochs': 50,
    'imgsz': 800,
    'batch': 2,
    'project': 'binary_detector',
    'max_det': 600,
    'dfl':3.5,
    'cls':0.8

}

# Save the hyperparameters to a YAML file
with open("cgs.yaml", "w") as file:
  yaml.dump(hyperparams, file)
print("Hyperparameters saved to 'cgs.yaml'.")

Hyperparameters saved to 'cgs.yaml'.


In [11]:

# Fine-tune YOLOv8 on binary dataset
#model = YOLO("/hpc/home/ama191/Our_Yolo_Solution/yolo11x-p2-coco/model (1).pt")
model = YOLO('/hpc/home/ama191/Our_Yolo_Solution/binary_detector/train31/weights/best.pt')  # You can change modl size
#model = YOLO("yolo11x-pose")

In [12]:
model.train(data=f"{dataset.location}/data.yaml",cfg="/hpc/home/ama191/Our_Yolo_Solution/cgs.yaml")


New https://pypi.org/project/ultralytics/8.3.157 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.152 🚀 Python-3.11.11 torch-2.7.0+cu126 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)


engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=2, bgr=0.0, box=7.5, cache=False, cfg=/hpc/home/ama191/Our_Yolo_Solution/cgs.yaml, classes=None, close_mosaic=10, cls=0.8, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/hpc/home/ama191/Our_Yolo_Solution/insects_noclasses-7/data.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=3.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=800, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=600, mixup=0.0, mode=train, model=/hpc/home/ama191/Our_Yolo_Solution/binary_detector/train31/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train32, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience

train: Scanning /hpc/home/ama191/Our_Yolo_Solution/insects_noclasses-7/train/labels.cache... 105 images, 21 backgrounds, 0 corrupt: 100%|██████████| 126/126 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.6±0.1 ms, read: 46.0±23.5 MB/s, size: 277.6 KB)


val: Scanning /hpc/home/ama191/Our_Yolo_Solution/insects_noclasses-7/valid/labels.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]


...

                   all          4       2444      0.644      0.559      0.606       0.24

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      50/50      7.77G      1.515      1.485      2.516         17        800: 100%|██████████| 63/63 [00:28<00:00,  2.19it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.53it/s]


                   all          4       2444      0.688      0.535      0.611      0.245

50 epochs completed in 0.455 hours.
Optimizer stripped from binary_detector/train32/weights/last.pt, 116.5MB
Optimizer stripped from binary_detector/train32/weights/best.pt, 116.5MB

Validating binary_detector/train32/weights/best.pt...
Ultralytics 8.3.152 🚀 Python-3.11.11 torch-2.7.0+cu126 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
                                                        CUDA:1 (Tesla P100-PCIE-16GB, 16269MiB)
YOLO11x-p2 summary (fused): 234 layers, 57,739,092 parameters, 0 gradients, 242.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.10s/it]


                   all          4       2444      0.678      0.561      0.618      0.253
Speed: 0.4ms preprocess, 494.7ms inference, 0.0ms loss, 6.0ms postprocess per image
Results saved to binary_detector/train32


In [21]:
model.val(save=True)

Ultralytics 8.3.152 🚀 Python-3.11.11 torch-2.7.0+cu126 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
                                                        CUDA:1 (Tesla P100-PCIE-16GB, 16269MiB)


YOLO11x-p2 summary (fused): 234 layers, 57,739,092 parameters, 0 gradients, 242.6 GFLOPs
val: Fast image access ✅ (ping: 0.7±0.1 ms, read: 134.8±30.0 MB/s, size: 277.6 KB)


val: Scanning /hpc/home/ama191/Our_Yolo_Solution/insects_noclasses-7/valid/labels.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:03<00:00,  1.54s/it]


                   all          4       2444      0.705      0.577       0.63      0.251
Speed: 1.2ms preprocess, 642.4ms inference, 0.0ms loss, 56.9ms postprocess per image
Results saved to binary_detector/train302


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7efcd17c00d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [18]:
# i have the model pre trained
model = YOLO("/content/binary_detector/train/weights/best.pt")

In [27]:

def crop_predictions(image_path, model, conf_thresh=0.01):
    results = model.predict(image_path, conf=conf_thresh,max_det=800)[0]
    img = cv2.imread(image_path)
    crops = []
    for box in results.boxes.xyxy.cpu().numpy():
        x1, y1, x2, y2 = map(int, box)
        crop = img[y1:y2, x1:x2]
        crops.append(crop)
    return crops


In [24]:
!pip install glob

ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob


In [28]:

# Load your trained binary detector
binary_detector =model
from glob import glob
import glob2

"""image_example = glob("W5_20220509_2_ST220182_A_JPG.rf.5d17c8a1b9b500761e1d6006c44d8dc9.jpg")
print(image_example)
cropped_images = crop_predictions(image_example[0], binary_detector)
"""
# Visualize crops
import matplotlib.pyplot as plt
import os
"""for i, crop in enumerate(cropped_images[:5]):  # visualize first 5 crops
    plt.figure()
    plt.imshow(cv2.cvtColor(crop, cv2.COLOR_BGR2RGB))
    plt.title(f"Cropped Image {i+1}")
    plt.axis('off')
    plt.show()
"""
image_paths = glob("/content/insects_noclasses-7/test/images/*.jpg")  # Adjust as needed
output_dir="/content/our_cropped2"
os.makedirs(output_dir, exist_ok=True)
# Loop through images, crop, and save
for image_path in image_paths:
    crops = crop_predictions(image_path, binary_detector)
    base_name = os.path.splitext(os.path.basename(image_path))[0]

    for idx, crop in enumerate(crops):
        save_path = os.path.join(output_dir, f"{base_name}_crop{idx}.jpg")
        cv2.imwrite(save_path, crop)

print(f"Saved all cropped images to: {output_dir}")


image 1/1 /content/insects_noclasses-7/test/images/W5_20230821_3_ST230743_B_JPG.rf.78954bd80d535b8aed00173c5773ed72.jpg: 1152x1504 18 Insects, 37.8ms
Speed: 12.3ms preprocess, 37.8ms inference, 1.9ms postprocess per image at shape (1, 3, 1152, 1504)

image 1/1 /content/insects_noclasses-7/test/images/W5_20220509_4_ST220184_A_JPG.rf.e4aa48e345f56ab1a719a223e7e08da4.jpg: 1504x1152 602 Insects, 38.2ms
Speed: 12.1ms preprocess, 38.2ms inference, 2.0ms postprocess per image at shape (1, 3, 1504, 1152)
Saved all cropped images to: /content/our_cropped2



## Next Steps:

- Apply augmentations on training cropped images. (per class)
- Feed cropped images into our classification model to fine tune it .
(Add your augmentation and classification inference code here.)

- after fine tuning our classification model we can run a total inference including both models



### Load Roboflow Dataset
Replace with your Roboflow API Key and Dataset Information.

In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key="TS1niacLXvvWTierCKCT")
project = rf.workspace("insectai").project("my-first-project-fmp4x")
dataset = project.version(12).download("yolov8")

print(f"Dataset location: {dataset.location}")






loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to My-First-Project-12 in yolov8:: 100%|██████████| 100/100 [00:00<00:00, 192.67it/s]


Dataset location: /hpc/home/ama191/My-First-Project-12


### Imports and Paths

In [ ]:
import cv2
import numpy as np
import os
from glob import glob
from tqdm import tqdm
from collections import Counter
import random

label_dir = os.path.join(dataset.location, "train", "labels")
image_dir = os.path.join(dataset.location, "train", "images")
cropped_output_dir = "cropped_dataset"
os.makedirs(cropped_output_dir, exist_ok=True)



### YOLO Conversion Functions

In [ ]:

def yolo_to_xyxy(bbox, img_w, img_h):
    x_c, y_c, w, h = bbox
    x1 = int((x_c - w / 2) * img_w)
    y1 = int((y_c - h / 2) * img_h)
    x2 = int((x_c + w / 2) * img_w)
    y2 = int((y_c + h / 2) * img_h)
    return x1, y1, x2, y2


### Image Augmentation

In [ ]:

def apply_augmentations(crop):
    angle = random.choice([0, 90, 180, 270])
    angle=angle + (np.random.random_integers(0,15))
    if angle:
        crop = np.rot90(crop, k=angle // 90)

    crop = cv2.GaussianBlur(crop, (5, 5), 0)
    noise = np.random.normal(0, 0.1, crop.shape).astype(np.uint8)
    crop = cv2.add(crop, noise)

    return crop


# Crop and Augment

In [ ]:

rare_classes = [0, 2,4]  # Modify as needed
augment_factor = 3

class_counter = Counter()

for label_file in tqdm(glob(f"{label_dir}/*.txt")):
    image_file = label_file.replace('/labels/', '/images/').replace('.txt', '.jpg')
    img = cv2.imread(image_file)
    h, w = img.shape[:2]

    with open(label_file, "r") as f:
        lines = f.readlines()

        for idx, line in enumerate(lines):
            parts = line.strip().split()
            class_id = int(parts[0])
            bbox = list(map(float, parts[1:]))
            x1, y1, x2, y2 = yolo_to_xyxy(bbox, w, h)

            crop = img[y1:y2, x1:x2]
            if crop.size == 0:
                continue

            class_folder = os.path.join(cropped_output_dir, str(class_id))
            os.makedirs(class_folder, exist_ok=True)

            filename = os.path.basename(image_file).replace('.jpg', f'_{idx}.jpg')
            cv2.imwrite(os.path.join(class_folder, filename), crop)
            class_counter[class_id] += 1

            if class_id in rare_classes:
                for aug_idx in range(augment_factor):
                    aug_crop = apply_augmentations(crop)
                    aug_filename = os.path.basename(image_file).replace('.jpg', f'_{idx}_aug{aug_idx}.jpg')
                    cv2.imwrite(os.path.join(class_folder, aug_filename), aug_crop)
                    class_counter[class_id] += 1

print("Class distribution after cropping and augmentation:", class_counter)


100%|██████████| 36/36 [00:40<00:00,  1.12s/it]

Class distribution after cropping and augmentation: Counter({1: 7631, 3: 667, 5: 287, 4: 132, 0: 48, 2: 12})


# Dataset Preparation for YOLO Classification

In [ ]:
!pip install sklearn
from sklearn.model_selection import train_test_split
import shutil

classification_dir = "classification_dataset"
os.makedirs(classification_dir, exist_ok=True)

for class_id in os.listdir(cropped_output_dir):
    class_path = os.path.join(cropped_output_dir, class_id)
    images = glob(os.path.join(class_path, '*.jpg'))

    train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42)

    for phase, img_paths in [('train', train_imgs), ('val', val_imgs)]:
        phase_class_dir = os.path.join(classification_dir, phase, class_id)
        os.makedirs(phase_class_dir, exist_ok=True)

        for img_path in img_paths:
            shutil.copy(img_path, phase_class_dir)

print("Dataset structured for YOLO classification at:", classification_dir)


Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... error
  ERROR: Command errored out with exit status 1:
   command: /usr/bin/python3 -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-vovrrbou/sklearn_660f25dc76c845e19dd404d51ca3bdee/setup.py'"'"'; __file__='"'"'/tmp/pip-install-vovrrbou/sklearn_660f25dc76c845e19dd404d51ca3bdee/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /tmp/pip-pip-egg-info-wih17095
       cwd: /tmp/pip-install-vovrrbou/sklearn_660f25dc76c845e19dd404d51ca3bdee/
  Complete output (15 lines):
  The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
  rather than 'sklearn' for pip commands.
  
  Here is how to fix this error 

# Train YOLO Classification Model

In [ ]:

!yolo task=classify mode=train model=yolov8s-cls.pt data={classification_dir} epochs=10 imgsz=224



100%|██████████████████████████████████████| 12.3M/12.3M [00:00<00:00, 72.7MB/s]
Ultralytics 8.3.146 🚀 Python-3.9.19 torch-2.7.0+cu126 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=classification_dataset, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=au

In [30]:

!yolo task=classify save_txt=True mode=predict model="/content/best_classifer_finetune_num2.pt" source={"/content/our_cropped2"}  imgsz=224



Ultralytics 8.3.156 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLOv8s-cls summary (fused): 30 layers, 5,082,886 parameters, 0 gradients, 12.5 GFLOPs

image 1/620 /content/our_cropped2/W5_20220509_4_ST220184_A_JPG.rf.e4aa48e345f56ab1a719a223e7e08da4_crop0.jpg: 224x224 0 0.38, 1 0.34, 3 0.22, 4 0.04, 5 0.02, 4.4ms
image 2/620 /content/our_cropped2/W5_20220509_4_ST220184_A_JPG.rf.e4aa48e345f56ab1a719a223e7e08da4_crop1.jpg: 224x224 1 0.63, 0 0.35, 3 0.02, 5 0.00, 2 0.00, 3.6ms
image 3/620 /content/our_cropped2/W5_20220509_4_ST220184_A_JPG.rf.e4aa48e345f56ab1a719a223e7e08da4_crop10.jpg: 224x224 1 0.56, 0 0.40, 3 0.04, 5 0.01, 2 0.00, 3.3ms
image 4/620 /content/our_cropped2/W5_20220509_4_ST220184_A_JPG.rf.e4aa48e345f56ab1a719a223e7e08da4_crop100.jpg: 224x224 1 0.52, 0 0.35, 3 0.13, 5 0.00, 2 0.00, 3.2ms
image 5/620 /content/our_cropped2/W5_20220509_4_ST220184_A_JPG.rf.e4aa48e345f56ab1a719a223e7e08da4_crop101.jpg: 224x224 0 0.36, 1 0.36, 3 0.28, 5 0.00, 4 0.0

In [ ]:
!pip install glob

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob


In [ ]:
import chardet

with open("/hpc/home/ama191/runs/classify/predict4/labels/W5_20220509_2_ST220182_A_JPG.rf.5d17c8a1b9b500761e1d6006c44d8dc9_crop0.txt", "rb") as f:
    raw = f.read()
    result = chardet.detect(raw)
    encoding = result['encoding']
    print(encoding)




ascii


In [31]:
import os
from collections import Counter

def count_top_predicted_classes(folder_path):
    """
    Reads all prediction .txt files in the folder and counts how often
    each class is the top (highest confidence) prediction in a file.

    Returns:
        Counter: A dictionary of class_id → count
    """
    top_class_counter = Counter()

    for file in os.listdir(folder_path):
        if file.endswith(".txt"):
            file_path = os.path.join(folder_path, file)
            top_conf = -1
            top_class = None

            with open(file_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) == 2:
                        conf = float(parts[0])
                        cls = int(parts[1])
                        if conf > top_conf:
                            top_conf = conf
                            top_class = cls

            if top_class is not None:
                top_class_counter[top_class] += 1

    return top_class_counter
count_top_predicted_classes("/content/runs/classify/predict/labels")

Counter({0: 78, 1: 531, 5: 7, 3: 4})